In [0]:
%matplotlib inline

In [0]:
from keras.models import load_model
from collections import deque
import numpy as np
import argparse
import pickle
import cv2

Using TensorFlow backend.


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
model = load_model('drive/My Drive/action_dataset/activity.model')
lb = pickle.loads(open('drive/My Drive/action_dataset/lb.pickle', 'rb').read())

W0802 16:39:47.381772 140091554023296 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0802 16:39:47.543303 140091554023296 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4185: The name tf.truncated_normal is deprecated. Please use tf.random.truncated_normal instead.

W0802 16:39:47.660668 140091554023296 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:245: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0802 16:39:47.667939 140091554023296 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0802 16:39:47.67290

In [0]:
mean = np.array([123.68, 116.779, 103.939], dtype="float32")

In [0]:
de = deque([])

In [0]:
writer = None
(W, H) = (None, None)
# print(cap)

In [0]:
from google.colab.patches import cv2_imshow
vs = cv2.VideoCapture('drive/My Drive/biking.mp4')
while(True):
  (grabbed, frame) = vs.read()
  if not grabbed:
    break
#   cv2.imshow('frame',frame)
#   if cv2.waitKey(20) & 0xFF == ord('q'):
#             break    
  if W is None or H is None:
    (H,W) = frame.shape[:2]
  output = frame.copy()
  frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
  frame = cv2.resize(frame, (224, 224)).astype('float32')
  frame -= mean
#   print(frame)
  preds = model.predict(np.expand_dims(frame, axis=0))[0]
  de.append(preds)
  results = np.array(de).mean(axis=0)
  i = np.argmax(results)
  label = lb.classes_[i]
#   print(frame)
  text = "activity: {}".format(label)
  cv2.putText(output, text, (35, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.25, (0, 255, 0), 5)
 
	# check if the video writer is None
  if writer is None:
		# initialize our video writer
    fourcc = cv2.VideoWriter_fourcc(*"MJPG")
    writer = cv2.VideoWriter('drive/My Drive/bikingoutcome.mp4', fourcc, 30,(W, H), True) 
	# write the output frame to disk
  writer.write(output)
 
	# show the output image
#   cv2_imshow("Output", output)
#   key = cv2.waitKey(1) & 0xFF
 
	# if the `q` key was pressed, break from the loop
#   if key == ord("q"):
#     break
 
# release the file pointers
print("[INFO] cleaning up...")
writer.release()
vs.release()

[INFO] cleaning up...
